<a href="https://colab.research.google.com/github/JeongHanJun/Colab/blob/master/Sequential_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# 순차 모델을 사용하는 경우
- 말그대로 순차 모델(Sequential Model) 이기 때문에, layer들을 스택으로 쌓는 경우에 적합하다.
- 입력과 출력이 텐서로 같은 경우

In [9]:
hj_model = keras.Sequential(
    [
     layers.Dense(2, activation = 'relu', name = 'layer1 with relu'),
     layers.Dense(3, activation = 'relu', name = 'layer2 with relu'),
     layers.Dense(4, name = 'layer3 without Activation Function')
    ]
)# 모델 생성자에 Layers 입력하여 모델 생성 , 해당 레이어들은 'hj_model.layers'로 조회 가능

# 모델에 layer 추가
hj_model.add(layers.Dense(2, activation = 'relu'))
#모델에 레이어를 제거
hj_model.pop()
x = tf.ones( (3, 3) )
y = hj_model(x)

In [12]:
x

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]], dtype=float32)>

In [13]:
y

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[-0.5403077 ,  0.342919  ,  0.5274871 ,  0.16128561],
       [-0.5403077 ,  0.342919  ,  0.5274871 ,  0.16128561],
       [-0.5403077 ,  0.342919  ,  0.5274871 ,  0.16128561]],
      dtype=float32)>

# 순차 모델이 적합하지 않은 경우
- 모델에 입력 또는 출력이 다중으로 존재할때 ( Multiple Input/Output in Model)
- 모든 레이어에 입력 또는 출력이 다중으로 존재할때( Multiple Input/Output in layers)
- 레이어 공유를 해야 하는 경우
- 비선형 토폴로지를 하는 경우 ( residual connectionn, multiple branch model)

In [14]:
hj_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer1 with relu (Dense)     multiple                  8         
_________________________________________________________________
layer2 with relu (Dense)     multiple                  9         
_________________________________________________________________
layer3 without Activation Fu multiple                  16        
Total params: 33
Trainable params: 33
Non-trainable params: 0
_________________________________________________________________


In [15]:
model = keras.Sequential()
model.add(keras.Input(shape = (4,)))
model.add(layers.Dense(2, activation = 'relu'))

In [16]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 2)                 10        
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.layers

In [18]:
model = keras.Sequential()
model.add(layers.Dense(2, activation = 'relu', input_shape = (4,)))

model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 2)                 10        
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


# 위 까지는 개념 설명이고, 실제로 모델을 어떻게 구성할 것인가?
- 일반적으로, add() 로 layer를 쌓고, summary()로 확인한다.

In [19]:
model = keras.Sequential()
model.add(keras.Input(shape = (250, 250, 3)))
model.add(layers.Conv2D(32, 5, strides = 2, activation = 'relu'))
model.add(layers.Conv2D(32, 3, activation = 'relu'))
model.add(layers.MaxPooling2D(3))

model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 123, 123, 32)      2432      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 121, 121, 32)      9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 40, 40, 32)        0         
Total params: 11,680
Trainable params: 11,680
Non-trainable params: 0
_________________________________________________________________


In [20]:
# 2개의 Layer를 쌓고, 1번의 Pooling , 이것은 일반적인 Sequential 모델의 아키텍쳐이다. 보통 이를 2~3번 반복하여 사용한다.
# 추가적으로 모델 내부에서는 relu, 마지막 출력적에서는 softmax 를 activation func로 사용한다.
model.add(layers.Conv2D(32, 3, activation = 'relu'))
model.add(layers.Conv2D(32, 3, activation = 'relu'))
model.add(layers.MaxPooling2D(3))

model.add(layers.Conv2D(32, 3, activation = 'relu'))
model.add(layers.Conv2D(32, 3, activation = 'relu'))
model.add(layers.MaxPooling2D(2))

model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 123, 123, 32)      2432      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 121, 121, 32)      9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 40, 40, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 38, 38, 32)        9248      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 36, 36, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 10, 10, 32)       

# 순차 모델을 사용한 특징 추출 ( Feature Selection wuth Sequential Model )

In [29]:
model_1 = keras.Sequential(
    [
     keras.Input(shape = (250, 250, 3)),
     layers.Conv2D(32, 5, strides = 2, activation = 'relu'),
     layers.Conv2D(32, 5, activation = 'relu'),
     layers.Conv2D(32, 3, activation = 'relu')
    ]
)

feature_extractor = keras.Model(
    inputs = model_1.inputs,
    outputs = [layer.output for layer in model_1.layers]
)

x = tf.ones((1, 250, 250, 3))
features = feature_extractor(x)

In [30]:
features

[<tf.Tensor: shape=(1, 123, 123, 32), dtype=float32, numpy=
 array([[[[0.        , 0.00929523, 0.00175618, ..., 0.09654894,
           0.29414874, 0.        ],
          [0.        , 0.00929523, 0.00175618, ..., 0.09654894,
           0.29414874, 0.        ],
          [0.        , 0.00929523, 0.00175618, ..., 0.09654894,
           0.29414874, 0.        ],
          ...,
          [0.        , 0.00929523, 0.00175618, ..., 0.09654894,
           0.29414874, 0.        ],
          [0.        , 0.00929523, 0.00175618, ..., 0.09654894,
           0.29414874, 0.        ],
          [0.        , 0.00929523, 0.00175618, ..., 0.09654894,
           0.29414874, 0.        ]],
 
         [[0.        , 0.00929523, 0.00175618, ..., 0.09654894,
           0.29414874, 0.        ],
          [0.        , 0.00929523, 0.00175618, ..., 0.09654894,
           0.29414874, 0.        ],
          [0.        , 0.00929523, 0.00175618, ..., 0.09654894,
           0.29414874, 0.        ],
          ...,
       

In [32]:
model_2 = keras.Sequential(
    [
     keras.Input(shape = (250, 250, 3)),
     layers.Conv2D(32, 5, strides = 2, activation = 'relu'),
     layers.Conv2D(32, 3, activation = 'relu', name = 'my_intermediate_layer'),
     layers.Conv2D(32, 3, activation = 'relu'),
    ]
)

feature_extractor = keras.Model(
    inputs = model_2.inputs,
    outputs = model_2.get_layer(name = 'my_intermediate_layer').output
)

x = tf.ones( (1, 250, 250, 3) )
features = feature_extractor(x)